In [ ]:
import csv
import os
import random
import pickle
import pandas as pd
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt
import sklearn
import warnings
import skbold
from skbold.preproc import ConfoundRegressor
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Define ConfoundRegressor
def confound_regressor(features_train, features_test, confounds_train, confounds_test):
    # Scale features (train and test sets)
    scaler_features = StandardScaler()
    features_train_scaled = scaler_features.fit_transform(features_train)
    features_test_scaled = scaler_features.transform(features_test)
    
    # Scale confounds (train and test sets)
    scaler_confounds = StandardScaler()
    confounds_train_scaled = scaler_confounds.fit_transform(confounds_train)
    confounds_test_scaled = scaler_confounds.transform(confounds_test)

    # Convert full sets into np.array
    features_full_scaled_np = np.array(pd.concat([pd.DataFrame(features_train_scaled, columns = features_train.columns), pd.DataFrame(features_test_scaled, columns = features_test.columns)], axis=0))
    confounds_full_scaled_np = np.array(pd.concat([pd.DataFrame(confounds_train_scaled, columns = confounds_train.columns), pd.DataFrame(confounds_test_scaled, columns = confounds_test.columns)], axis=0))
    
    # Define ConfoundRegressor on a FULL set (train and test)
    cfr = ConfoundRegressor(confound=confounds_full_scaled_np, X=features_full_scaled_np)
    features_train_corrected = cfr.fit_transform(features_train_scaled)
    features_test_corrected = cfr.transform(features_test_scaled)


    return features_train_corrected, features_test_corrected, features_train_scaled, features_test_scaled, scaler_features, scaler_confounds

## Define modalities
modalities = [
#31020
'31020_connectome_fa',
'31020_connectome_mean_length',
'31020_connectome_sift2',
'31020_connectome_streamline_count',
#31021
'31021_connectome_fa',
'31021_connectome_mean_length',
'31021_connectome_sift2',
'31021_connectome_streamline_count',
#31022
'31022_connectome_fa',
'31022_connectome_mean_length',
'31022_connectome_sift2',
'31022_connectome_streamline_count',
#31023
'31023_connectome_fa',
'31023_connectome_mean_length',
'31023_connectome_sift2',
'31023_connectome_streamline_count',
#31024
'31024_connectome_fa',
'31024_connectome_mean_length',
'31024_connectome_sift2',
'31024_connectome_streamline_count',
#31025
'31025_connectome_fa',
'31025_connectome_mean_length',
'31025_connectome_sift2',
'31025_connectome_streamline_count']

confounds = pd.read_csv('/PLS/pls_dti/dti_confounds_final_set.csv')

warnings.simplefilter(action='ignore', category=FutureWarning)

############## 1
print('Uploading brain data')
seed = 42

for modality in modalities:

    file = pd.read_csv(f'/PLS/pls_dti/structural_files/{modality}.csv').rename(columns={'ID': 'eid'})
    

    folds = ["0", "1", "2", "3", "4"]
    pls_result = {}
    
    # Match confounds to MRI
    print(f'Matching brain data to confounds in {modality}', flush=True)
    conf_to_brain_match = pd.merge(confounds, file['eid'], on='eid')
    brain_to_conf_match = pd.merge(conf_to_brain_match['eid'], file, on='eid')

    for fold in folds:
        test_id = pd.read_csv(f'/PLS/pls_dti/fold_id/test_id_fold_{fold}.csv')
        train_id = pd.read_csv(f'/PLS/pls_dti/fold_id/train_id_fold_{fold}.csv')
        
        # Upload g-factor with ID
        g_train_full = pd.read_csv(f'/PLS/pls_dti/g_factor/g_train_with_id_fold_{fold}.csv')
        g_test_full = pd.read_csv(f'/PLS/pls_dti/g_factor/g_test_with_id_fold_{fold}.csv')
    
        # Match brain data to cognitive data
        print(f'Matching brain data to confounds in {modality}', flush=True)
        brain_train, brain_test, brain_train_id, brain_test_id = pd.merge(brain_to_conf_match, train_id, on='eid').drop(columns=['eid']), pd.merge(brain_to_conf_match, test_id, on='eid').drop(columns=['eid']), pd.merge(brain_to_conf_match, train_id, on='eid')['eid'], pd.merge(brain_to_conf_match, test_id, on='eid')['eid']

        brain_train.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_train_fold_{fold}.csv', index=False)
        brain_test.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_test_fold_{fold}.csv', index=False)
        brain_train_id.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_train_id_fold_{fold}.csv', index=False)
        brain_test_id.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_test_id_fold_{fold}.csv', index=False)
        
        ############## 2
        print(f'Matching confounds to {modality} fold {fold}', flush=True)
        
        # Match confounds to MRI
        brain_conf_train, brain_conf_test = pd.merge(conf_to_brain_match, brain_train_id, on='eid').drop(columns=['eid']), pd.merge(conf_to_brain_match, brain_test_id, on='eid').drop(columns=['eid'])
        brain_conf_train.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_conf_train_fold_{fold}.csv', index=False)
        brain_conf_test.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/{modality}_conf_test_fold_{fold}.csv', index=False)
        
        ############## 3
        print(f'Matching g-factor to {modality} fold {fold}', flush=True)
        
        # Match g-factor back to DTI
        g_train, g_test, g_train_id, g_test_id = pd.merge(g_train_full, brain_train_id, on='eid').drop(columns=['eid']), pd.merge(g_test_full, brain_test_id, on='eid').drop(columns=['eid']), pd.merge(g_train_full, brain_train_id, on='eid')['eid'], pd.merge(g_test_full, brain_test_id, on='eid')['eid']
        g_train.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/g_train_{modality}_matched_fold_{fold}.csv', index=False)
        g_test.to_csv(f'/PLS/pls_dti/fold_{fold}/suppl/g_test_{modality}_matched_fold_{fold}.csv', index=False)
        
        ############## 4
        print(f'Applying ConfoundRegressor to {modality} fold {fold}', flush=True)
        
        # Apply ConfoundRegressor
        features_train_corr, features_test_corr, features_train_scaled, features_test_scaled, scaler_features, scaler_confounds = confound_regressor(brain_train, brain_test, brain_conf_train, brain_conf_test)
        pd.DataFrame(features_train_corr, columns = brain_train.columns).to_csv(f'/PLS/pls_dti/fold_{fold}/scaling/{modality}_train_corr_{fold}.csv', index=False)
        pd.DataFrame(features_test_corr, columns = brain_test.columns).to_csv(f'/PLS/pls_dti/fold_{fold}/scaling/{modality}_test_corr_{fold}.csv', index=False)

        pd.DataFrame(features_train_scaled, columns = brain_train.columns).to_csv(f'/PLS/pls_dti/fold_{fold}/scaling/{modality}_train_scaled_{fold}.csv', index=False)
        pd.DataFrame(features_test_scaled, columns = brain_test.columns).to_csv(f'/PLS/pls_dti/fold_{fold}/scaling/{modality}_test_scaled_{fold}.csv', index=False)

        
        with open(f'/PLS/pls_dti/fold_{fold}/scaling/scaler_features_{modality}_fold_{fold}.pkl', "wb") as f:
            pickle.dump(scaler_features, f)

        with open(f'/PLS/pls_dti/fold_{fold}/scaling/scaler_confounds_{modality}_fold_{fold}.pkl', "wb") as f:
            pickle.dump(scaler_confounds, f)

        # Initiate and run PLS
        parameters = {'n_components': range(1, 30, 1)}
        pls = PLSRegression()
        model = GridSearchCV(pls, parameters, scoring = 'neg_mean_absolute_error', cv=KFold(10, shuffle = True, random_state=seed), verbose=4)
        
        
        print(f'Fitting PLS to {modality} fold {fold}', flush=True)
        model.fit(features_train_corr, np.array(g_train))
        
        print(f'Model parameters for fold {fold}:', model.cv_results_['params'])
        print(f'Mean test score for fold {fold}:', model.cv_results_['mean_test_score'])
        print(f'Rank test score for fold {fold}:', model.cv_results_['rank_test_score'])
        print(model)
        
        print(f'Saving PLS model for {modality} fold {fold}', flush=True)
        with open(f'/PLS/pls_dti/fold_{fold}/models/pkl/{modality}_model_fold_{fold}.pkl', "wb") as f:
            pickle.dump(model, f)
            
        print(f'Best params in fold {fold} = ', model.best_params_)
        print(f'Best score (neg_mean_absolute_error) in fold {fold} = ', model.best_score_)
            
        # Predict the values
        print(f'Predicting & saving g_test for {modality} fold {fold}', flush=True)
        g_pred_test = model.predict(np.array(features_test_corr))
        pd.DataFrame(g_pred_test, columns=['g predicted test']).to_csv(f'/PLS/pls_dti/fold_{fold}/g_pred/{modality}_test_fold_{fold}.csv')

        g_pred_test_with_id = pd.concat([g_test_id.astype(int), pd.DataFrame(g_pred_test, columns=['g predicted test'])], axis=1).to_csv(f'/PLS/pls_dti/fold_{fold}/g_pred/{modality}_g_pred_test_id_fold_{fold}.csv')

        
        print(f'Predicting & saving g_train for {modality} fold {fold}', flush=True)
        g_pred_train = model.predict(np.array(features_train_corr))
        pd.DataFrame(g_pred_train, columns=['g predicted train']).to_csv(f'/PLS/pls_dti/fold_{fold}/g_pred/{modality}_g_pred_train_fold_{fold}.csv')
        

        g_pred_train_with_id = pd.concat([g_train_id.astype(int), pd.DataFrame(g_pred_train, columns=['g predicted train'])], axis=1).to_csv(f'/PLS/pls_dti/fold_{fold}/g_pred/{modality}_g_pred_train_id_fold_{fold}.csv')
        
            
        print(f"Fold = {fold}")
        print("----------")
        print("MSE = ", mean_squared_error(np.array(g_test)[:,0], g_pred_test[:,0]))
        print("MAE = ", mean_absolute_error(np.array(g_test)[:,0], g_pred_test[:,0]))
        print("R2 = ", r2_score(np.array(g_test)[:,0], g_pred_test[:,0]))
        print("Pearson's r = ", pearsonr(np.array(g_test)[:,0], g_pred_test[:,0]))
        print("----------")
            
        pls_result['fold'] = fold
        pls_result['modality'] = modality
        pls_result['n_components'] = model.best_params_
        pls_result['MSE'] = mean_squared_error(np.array(g_test)[:,0], g_pred_test[:,0])
        pls_result['MAE'] = mean_absolute_error(np.array(g_test)[:,0], g_pred_test[:,0])
        pls_result['R2'] = r2_score(np.array(g_test)[:,0], g_pred_test[:,0])
        pls_result['Pearson r'] = pearsonr(np.array(g_test)[:,0], g_pred_test[:,0])
            
        with open(f'/PLS/pls_dti/fold_{fold}/models/csv/{modality}_fold_{fold}_PLS_result.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=pls_result.keys())
            writer.writerow(pls_result)
            
        pls_result.clear()
        
        corr, pval = stats.pearsonr(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
        r2 = r2_score(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
        mse = mean_squared_error(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
        result = pd.DataFrame([modality, fold, corr, pval, r2, mse, model.best_params_], index=['Modality', 'Fold', 'Correlation', 'P-value', 'R2', 'MSE', 'n components'], columns=['Values']).to_csv(f'/PLS/pls_dti/fold_{fold}/models/csv/{modality}_fold_{fold}_full_result.csv')



Below is the script that was run on the clusted via slurm

In [ ]:
# %%
import csv
import os
import random
import pickle
import pandas as pd
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt
import sklearn
import warnings
import sys
import skbold
from skbold.preproc import ConfoundRegressor
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# %%
## Define ConfoundRegressor
def confound_regressor(features_train, features_test, confounds_train, confounds_test):
    # Scale features (train and test sets)
    scaler_features = StandardScaler()
    features_train_scaled = scaler_features.fit_transform(features_train)
    features_test_scaled = scaler_features.transform(features_test)
    
    # Scale confounds (train and test sets)
    scaler_confounds = StandardScaler()
    confounds_train_scaled = scaler_confounds.fit_transform(confounds_train)
    confounds_test_scaled = scaler_confounds.transform(confounds_test)

    # Convert full sets into np.array
    features_full_scaled_np = np.array(pd.concat([pd.DataFrame(features_train_scaled, columns = features_train.columns), pd.DataFrame(features_test_scaled, columns = features_test.columns)], axis=0))
    confounds_full_scaled_np = np.array(pd.concat([pd.DataFrame(confounds_train_scaled, columns = confounds_train.columns), pd.DataFrame(confounds_test_scaled, columns = confounds_test.columns)], axis=0))
    
    # Define ConfoundRegressor on a FULL set (train and test)
    cfr = ConfoundRegressor(confound=confounds_full_scaled_np, X=features_full_scaled_np)
    features_train_corrected = cfr.fit_transform(features_train_scaled)
    features_test_corrected = cfr.transform(features_test_scaled)


    return features_train_corrected, features_test_corrected, features_train_scaled, features_test_scaled, scaler_features, scaler_confounds

# %%
## Define modalities
modalities = [
'31025_connectome_fa',
'31025_connectome_mean_length',
'31025_connectome_sift2',
'31025_connectome_streamline_count']

folds = ["0", "1", "2", "3", "4"]

###################################################### Preparatory steps

warnings.simplefilter(action='ignore', category=FutureWarning)
seed = 42
pls_result = {}

if len(sys.argv) > 1:
    fold = int(sys.argv[1]) % 5
    modal = int(sys.argv[1]) // 5

print('Reading confounds', flush=True)
confounds = pd.read_csv('/PLS/pls_dti/dti_confounds_final_set.csv')

warnings.simplefilter(action='ignore', category=FutureWarning)

############## 1
print(f'Reading {modalities[modal]}', flush=True)
file = pd.read_csv(f'/PLS/pls_dti/structural_files/{modalities[modal]}.csv').rename(columns={'ID': 'eid'})
    
 
# Match confounds to MRI
print(f'Matching brain data to confounds in {modalities[modal]}', flush=True)
conf_to_brain_match = pd.merge(confounds, file['eid'], on='eid')
brain_to_conf_match = pd.merge(conf_to_brain_match['eid'], file, on='eid')


test_id = pd.read_csv(f'/PLS/pls_dti/fold_id/test_id_fold_{folds[fold]}.csv')
train_id = pd.read_csv(f'/PLS/pls_dti/fold_id/train_id_fold_{folds[fold]}.csv')

# Upload g-factor with ID
g_train_full = pd.read_csv(f'/PLS/pls_dti/g_factor/g_train_with_id_fold_{folds[fold]}.csv')
g_test_full = pd.read_csv(f'/PLS/pls_dti/g_factor/g_test_with_id_fold_{folds[fold]}.csv')
    
# Match brain data to cognitive data
print(f'Matching brain data to confounds in {modalities[modal]}', flush=True)
brain_train, brain_test, brain_train_id, brain_test_id = pd.merge(brain_to_conf_match, train_id, on='eid').drop(columns=['eid']), pd.merge(brain_to_conf_match, test_id, on='eid').drop(columns=['eid']), pd.merge(brain_to_conf_match, train_id, on='eid')['eid'], pd.merge(brain_to_conf_match, test_id, on='eid')['eid']

brain_train.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_train_fold_{folds[fold]}.csv', index=False)
brain_test.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_test_fold_{folds[fold]}.csv', index=False)
brain_train_id.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_train_id_fold_{folds[fold]}.csv', index=False)
brain_test_id.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_test_id_fold_{folds[fold]}.csv', index=False)

############## 2
print(f'Matching confounds to {modalities[modal]} fold {folds[fold]}', flush=True)

# Match confounds to MRI
brain_conf_train, brain_conf_test = pd.merge(conf_to_brain_match, brain_train_id, on='eid').drop(columns=['eid']), pd.merge(conf_to_brain_match, brain_test_id, on='eid').drop(columns=['eid'])
brain_conf_train.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_conf_train_fold_{folds[fold]}.csv', index=False)
brain_conf_test.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/{modalities[modal]}_conf_test_fold_{folds[fold]}.csv', index=False)

############## 3
print(f'Matching g-factor to {modalities[modal]} fold {folds[fold]}', flush=True)

# Match g-factor back to DTI
g_train, g_test, g_train_id, g_test_id = pd.merge(g_train_full, brain_train_id, on='eid').drop(columns=['eid']), pd.merge(g_test_full, brain_test_id, on='eid').drop(columns=['eid']), pd.merge(g_train_full, brain_train_id, on='eid')['eid'], pd.merge(g_test_full, brain_test_id, on='eid')['eid']
g_train.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/g_train_{modalities[modal]}_matched_fold_{folds[fold]}.csv', index=False)
g_test.to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/suppl/g_test_{modalities[modal]}_matched_fold_{folds[fold]}.csv', index=False)

############## 4
print(f'Applying ConfoundRegressor to {modalities[modal]} fold {folds[fold]}', flush=True)

# Apply ConfoundRegressor
features_train_corr, features_test_corr, features_train_scaled, features_test_scaled, scaler_features, scaler_confounds = confound_regressor(brain_train, brain_test, brain_conf_train, brain_conf_test)
pd.DataFrame(features_train_corr, columns = brain_train.columns).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/{modalities[modal]}_train_corr_{folds[fold]}.csv', index=False)
pd.DataFrame(features_test_corr, columns = brain_test.columns).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/{modalities[modal]}_test_corr_{folds[fold]}.csv', index=False)

pd.DataFrame(features_train_scaled, columns = brain_train.columns).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/{modalities[modal]}_train_scaled_{folds[fold]}.csv', index=False)
pd.DataFrame(features_test_scaled, columns = brain_test.columns).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/{modalities[modal]}_test_scaled_{folds[fold]}.csv', index=False)


with open(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/scaler_features_{modalities[modal]}_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_features, f)

with open(f'/PLS/pls_dti/fold_{folds[fold]}/scaling/scaler_confounds_{modalities[modal]}_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(scaler_confounds, f)

# Initiate and run PLS
parameters = {'n_components': range(1, 30, 1)}
pls = PLSRegression()
model = GridSearchCV(pls, parameters, scoring = 'neg_mean_absolute_error', cv=KFold(10, shuffle = True, random_state=seed), verbose=4)


print(f'Fitting PLS to {modalities[modal]} fold {folds[fold]}', flush=True)
model.fit(features_train_corr, np.array(g_train))

print(f'Model parameters for fold {folds[fold]}:', model.cv_results_['params'])
print(f'Mean test score for fold {folds[fold]}:', model.cv_results_['mean_test_score'])
print(f'Rank test score for fold {folds[fold]}:', model.cv_results_['rank_test_score'])
print(model)

print(f'Saving PLS model for {modalities[modal]} fold {folds[fold]}', flush=True)
with open(f'/PLS/pls_dti/fold_{folds[fold]}/models/pkl/{modalities[modal]}_model_fold_{folds[fold]}.pkl', "wb") as f:
    pickle.dump(model, f)
    
print(f'Best params in fold {folds[fold]} = ', model.best_params_)
print(f'Best score (neg_mean_absolute_error) in fold {folds[fold]} = ', model.best_score_)
    
# Predict the values
print(f'Predicting & saving g_test for {modalities[modal]} fold {folds[fold]}')
g_pred_test = model.predict(np.array(features_test_corr))
pd.DataFrame(g_pred_test, columns=['g predicted test']).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/g_pred/{modalities[modal]}_test_fold_{folds[fold]}.csv')

g_pred_test_with_id = pd.concat([g_test_id.astype(int), pd.DataFrame(g_pred_test, columns=['g predicted test'])], axis=1).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/g_pred/{modalities[modal]}_g_pred_test_id_fold_{folds[fold]}.csv')


print(f'Predicting & saving g_train for {modalities[modal]} fold {folds[fold]}', flush=True)
g_pred_train = model.predict(np.array(features_train_corr))
pd.DataFrame(g_pred_train, columns=['g predicted train']).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/g_pred/{modalities[modal]}_g_pred_train_fold_{folds[fold]}.csv')


g_pred_train_with_id = pd.concat([g_train_id.astype(int), pd.DataFrame(g_pred_train, columns=['g predicted train'])], axis=1).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/g_pred/{modalities[modal]}_g_pred_train_id_fold_{folds[fold]}.csv')

    
print(f"Fold = {folds[fold]}")
print("----------")
print("MSE = ", mean_squared_error(np.array(g_test)[:,0], g_pred_test[:,0]))
print("MAE = ", mean_absolute_error(np.array(g_test)[:,0], g_pred_test[:,0]))
print("R2 = ", r2_score(np.array(g_test)[:,0], g_pred_test[:,0]))
print("Pearson's r = ", pearsonr(np.array(g_test)[:,0], g_pred_test[:,0]))
print("----------")
    
pls_result['fold'] = folds[fold]
pls_result['modality'] = modalities[modal]
pls_result['n_components'] = model.best_params_
pls_result['MSE'] = mean_squared_error(np.array(g_test)[:,0], g_pred_test[:,0])
pls_result['MAE'] = mean_absolute_error(np.array(g_test)[:,0], g_pred_test[:,0])
pls_result['R2'] = r2_score(np.array(g_test)[:,0], g_pred_test[:,0])
pls_result['Pearson r'] = pearsonr(np.array(g_test)[:,0], g_pred_test[:,0])
    
with open(f'/PLS/pls_dti/fold_{folds[fold]}/models/csv/{modalities[modal]}_fold_{folds[fold]}_PLS_result.csv', 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=pls_result.keys())
    writer.writerow(pls_result)
    
pls_result.clear()

corr, pval = stats.pearsonr(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
r2 = r2_score(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
mse = mean_squared_error(np.squeeze(np.array(g_test)), np.squeeze(g_pred_test))
result = pd.DataFrame([modalities[modal], folds[fold], corr, pval, r2, mse, model.best_params_], index=['Modality', 'Fold', 'Correlation', 'P-value', 'R2', 'MSE', 'n components'], columns=['Values']).to_csv(f'/PLS/pls_dti/fold_{folds[fold]}/models/csv/{modalities[modal]}_fold_{folds[fold]}_full_result.csv')